In [1]:
%cd ../..

/home/andrea/Projects/piacere-model-checker


In [11]:
%pip install diagrams

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 14.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 860.3/860.3 kB 14.2 MB/s eta 0:00:0031m15.1 MB/s eta 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [46]:
XMI_PATH = "./tests/doml/CaseStudies/posidonia.domlx"

with open(XMI_PATH, 'rb') as f:
    doml_xmi = f.read()

In [47]:
from mc_openapi.doml_mc.intermediate_model.metamodel import DOMLVersion
from mc_openapi.doml_mc.main import init_model

dmc = init_model(doml_xmi, doml_ver=None)

[x for x in dmc.intermediate_model.values() if x.user_friendly_name and x.user_friendly_name.startswith("co1")]

[]

In [94]:
from diagrams import Diagram, Cluster, Node, Edge
from diagrams.aws.compute import EC2, ECS, AutoScaling, AMI, Compute, EC2Instance, AppRunner
from diagrams.aws.network import CloudFront, APIGateway
from diagrams.aws.storage import S3
from diagrams.aws.security import Shield, IAMPermissions, IdentityAndAccessManagementIamAddOn as KeyPair
from diagrams.aws.general import General
from diagrams.aws.database import DB
from diagrams.aws.mobile import APIGateway

from mc_openapi.doml_mc.intermediate_model.doml_element import DOMLElement

elems = [elem for elem in dmc.intermediate_model.values()]

common_elems = [e for e in elems if e.class_.startswith("commons_") if e.class_ not in ["commons_Deployment", "commons_Configuration"]] 
deployments = [e for e in elems if e.class_ == "commons_Deployment"] 
app_elems = [e for e in elems if e.class_.startswith("application_") and e.class_ != "application_ApplicationLayer"] 
infr_elems = [e for e in elems if e.class_.startswith("infrastructure_") and e.class_ != "infrastructure_InfrastructureLayer"] 
concr_elems = [e for e in elems if e.class_.startswith("concrete_") and e.class_ != "concrete_ConcreteInfrastructure"] 

group_elems = {"Application": app_elems, "Infrastructure": infr_elems, "Concretization": concr_elems}

def prettify_class_name(name: str):
    _, name = name.split('_', 1)
    return name

def prettify_assoc_name(assoc: str):
    _, assoc = assoc.split('::', 1)
    return assoc

def get_component(elem: DOMLElement):
    
    class_name = prettify_class_name(elem.class_)

    name = f"{elem.user_friendly_name or ''}\n({class_name})"

    match class_name:
        case "SoftwareComponent":
            return Compute(name)
        case "SoftwareInterface":
            return APIGateway(name)
        case "DBMS":
            return DB(name)
        case "Network":
            if address := elem.attributes.get('infrastructure_Network::cidr'):
                name += f'\nCIDR: {address[0]}'
            return CloudFront(name)
        case "Subnet":
            if address := elem.attributes.get('infrastructure_Network::cidr'):
                name += f'\nCIDR: {address[0]}'
            return CloudFront(name)
        case "NetworkInterface":
            if address := elem.attributes.get('infrastructure_NetworkInterface::endPoint'):
                name += f'\nIP: {address[0]}'
            return APIGateway(name)
        case "Storage":
            return S3(name)
        case "VirtualMachine":
            return EC2(name)
        case "Container":
            return ECS(name)
        case "VMImage":
            return AMI(name)
        case "AutoScalingGroup":
            return AutoScaling(name)
        case "ComputingGroup":
            return EC2Instance(name)
        case "RuntimeProvider":
            return AppRunner(name)
        case "KeyPair":
            return KeyPair(name)
        case "UserPass":
            return KeyPair(name)
        case "SecurityGroup":
            return Shield(name)
        case "Rule":
            return IAMPermissions(name)
        case _:
            return General(name)
        
graph_attr = {
    "pad": "1pt"
}

HIGHLIGHT_CONNECTIONS = [
    'infrastructure_NetworkInterface::belongsTo'
]

with Diagram("DOML Model", show=True, outformat="svg", direction="TB", graph_attr=graph_attr) as diagram:
    cached_nodes: dict[str, Node] = {}
    for gk, gv in group_elems.items():
        with Cluster(gk):
            cached_nodes |= {e.id_: get_component(e) for e in gv}
    
    # Dont group common elements
    cached_nodes |= {e.id_: get_component(e) for e in common_elems}

    for groups in group_elems.values():
        for elem in groups:
            for assoc_k, assocs_v in elem.associations.items():
                for assoc in assocs_v:
                    if (start := cached_nodes.get(elem.id_)) and (end := cached_nodes.get(assoc)):
                        if assoc_k in HIGHLIGHT_CONNECTIONS:
                                start >> Edge(label=prettify_assoc_name(assoc_k), color='green', style="bold") >> end
                        elif 'maps' in assoc_k:
                            start >> Edge(label=prettify_assoc_name(assoc_k), style="bold") >> end
                        else:
                            start >> end

    for dep in deployments:
        for _component in dep.associations.get('commons_Deployment::component'):
            for _node in dep.associations.get('commons_Deployment::node'):
                 if (start := cached_nodes.get(_component)) and (end := cached_nodes.get(_node)):
                        start >> Edge(style="bold") >> end
